In [13]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Loading the dataset and preprocessing

In [14]:
import pandas as pd
from transformers import TapasTokenizer, TapasForQuestionAnswering
import pandas as pd

df=pd.read_csv("/kaggle/input/students-grading-dataset/Students Performance Dataset.csv")

In [15]:
df.isnull().sum()


Student_ID                       0
First_Name                       0
Last_Name                        0
Email                            0
Gender                           0
Age                              0
Department                       0
Attendance (%)                   0
Midterm_Score                    0
Final_Score                      0
Assignments_Avg                  0
Quizzes_Avg                      0
Participation_Score              0
Projects_Score                   0
Total_Score                      0
Grade                            0
Study_Hours_per_Week             0
Extracurricular_Activities       0
Internet_Access_at_Home          0
Parent_Education_Level        1025
Family_Income_Level              0
Stress_Level (1-10)              0
Sleep_Hours_per_Night            0
dtype: int64

In [16]:
df=df.drop('Parent_Education_Level',axis=1)

In [26]:
df.sample(4)


,Student_ID,First_Name,Last_Name,Email,Gender,Age,Department,Attendance (%),Midterm_Score,Final_Score,...,Projects_Score,Total_Score,Grade,Study_Hours_per_Week,Extracurricular_Activities,Internet_Access_at_Home,Family_Income_Level,Stress_Level (1-10),Sleep_Hours_per_Night,embeddings
2471,S3471,Sara,Williams,student2471@university.com,Male,18,CS,76.87,51.42,91.27,...,85.81,76.9435,C,25.3,No,Yes,Medium,3,7.1,"[-0.14438328, -0.07110767, -0.01676564, -0.023..."
2316,S3316,Sara,Williams,student2316@university.com,Male,21,Mathematics,55.48,82.42,58.38,...,83.57,77.1225,C,26.5,No,No,High,8,7.9,"[-0.12593693, -0.0074540856, -0.031417027, -0...."
1566,S2566,Ahmed,Johnson,student1566@university.com,Male,22,Engineering,67.43,86.26,42.24,...,93.66,70.906,C,12.6,Yes,No,Medium,4,6.9,"[-0.044818163, 0.060231566, -0.07624617, 0.019..."
1128,S2128,Emma,Johnson,student1128@university.com,Male,21,Mathematics,87.46,43.88,71.13,...,98.21,73.66399999999999,C,11.5,Yes,No,Medium,8,8.3,"[-0.124923356, -0.1040622, 0.023071107, -0.020..."


# Creating vector database

In [19]:
data = df
table = pd.DataFrame(data)

df= table.astype(str)


In [30]:

pip install faiss-cpu



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('all-MiniLM-L6-v2')
texts = df.values.tolist()
embeddings = model.encode(texts, show_progress_bar=True)
df['embeddings'] = list(embeddings)


query = "What is Omar's last name?"

query_embedding = model.encode([query])[0]


similarities = cosine_similarity([query_embedding], embeddings)[0]


top_k = 3
top_k_indices = np.argsort(similarities)[::-1][:top_k]
top_k_scores = similarities[top_k_indices]


top_k_df = df.iloc[top_k_indices].copy()
top_k_df['similarity_score'] = top_k_scores


print("Query:", query)
print("\nTop-3 relevant rows:")
top_k_df.head()

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: What is Omar's last name?

Top-3 relevant rows:


,Student_ID,First_Name,Last_Name,Email,Gender,Age,Department,Attendance (%),Midterm_Score,Final_Score,...,Total_Score,Grade,Study_Hours_per_Week,Extracurricular_Activities,Internet_Access_at_Home,Family_Income_Level,Stress_Level (1-10),Sleep_Hours_per_Night,embeddings,similarity_score
2357,S3357,Omar,Johnson,student2357@university.com,Male,23,Business,62.39,87.48,41.28,...,74.0135,C,18.3,No,No,Medium,2,7.7,"[-0.10489333, 0.030186659, -0.048220698, 0.087...",0.658451
1042,S2042,Omar,Johnson,student1042@university.com,Female,22,CS,72.94,63.67,50.96,...,60.2605,D,25.3,Yes,Yes,Medium,7,6.8,"[-0.061536636, 0.06521622, -0.053399332, 0.091...",0.657086
921,S1921,Omar,Williams,student921@university.com,Female,20,Business,66.71,97.09,90.64,...,80.81649999999999,B,13.0,No,Yes,High,10,8.2,"[-0.08745918, 0.0142855635, -0.07929673, 0.060...",0.649887


# Buillding model 

#### "google/tapas-large-finetuned-wtq"

In [23]:
def daddy_answer_please(query):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  texts = df.values.tolist()
  embeddings = model.encode(texts, show_progress_bar=True)
  df['embeddings'] = list(embeddings)
  query = query
  query_embedding = model.encode([query])[0]
  similarities = cosine_similarity([query_embedding], embeddings)[0]
  top_k = 3
  top_k_indices = np.argsort(similarities)[::-1][:top_k]
  top_k_scores = similarities[top_k_indices]
  top_k_df = df.iloc[top_k_indices].copy()
  top_k_df['similarity_score'] = top_k_scores
  model_name = "google/tapas-large-finetuned-wtq"
  tokenizer = TapasTokenizer.from_pretrained(model_name)
  model = TapasForQuestionAnswering.from_pretrained(model_name)




  table=top_k_df
  table = table.astype(str)
  table.reset_index(drop=True, inplace=True)

  inputs = tokenizer(
    table=table,
    queries=query,
    padding="max_length",
    return_tensors="pt",
    max_length=512,
    truncation=True
   )


  outputs = model(**inputs)


  predicted_answer_coordinates, predicted_aggregation_indices = tokenizer.convert_logits_to_predictions(
    inputs,
    outputs.logits.detach(),
    outputs.logits_aggregation.detach()
   )


  answer_coordinates = predicted_answer_coordinates[0]
  answer_cells = [table.iloc[coords[0], coords[1]] for coords in answer_coordinates]


  print(f"Query: {query}")
  if answer_cells:
    print(f"Answer: {answer_cells}")
  else:
    print("Answer: The model can't answer that.")



# Testing the model

In [25]:
daddy_answer_please("what is the Department of student id:S1000?")


Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: what is the Department of student id:S1000?
Answer: ['Business']
